# STEREO/SECCHI-EUVI

In [ ]:
ds_path = "./data/stereo/euvi"

## explore

In [ ]:
from pathlib import Path

wavelengths = [171, 195, 284, 304]
dirs = [str(wl) for wl in wavelengths]

[(Path(ds_path) / "a" / wl).mkdir(parents=True, exist_ok=True) for wl in dirs]
[(Path(ds_path) / "b" / wl).mkdir(parents=True, exist_ok=True) for wl in dirs]

[None, None, None, None]

In [ ]:
import re
import astropy.units as u
from datetime import timedelta

def parse_interval(interval):
    match = re.match(r"(\d+)\s*(\D+)", interval)
    if match:
        value = int(match.group(1))
        unit = match.group(2).strip()
    return value, unit

def get_timedelta(interval):
    value, unit = parse_interval(interval)
    if unit == "days":
        return timedelta(days=value)
    elif unit == "hours":
        return timedelta(hours=value)
    elif unit == "minutes":
        return timedelta(minutes=value)
    else:
        raise ValueError("Invalid unit. Only days, hours, and minutes are supported.")
    
def get_astropy_time(interval):
    value, unit = parse_interval(interval)
    if unit == "days":
        return value * u.day
    elif unit == "hours":
        return value * u.hour
    elif unit == "minutes":
        return value * u.minute
    else:
        raise ValueError("Invalid unit. Only days, hours, and minutes are supported.")

### SunPy

In [ ]:
start_date = "2011-01-01"
end_date = "2011-01-02"

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sunpy.net import Fido, attrs as a

t_start = datetime.strptime(start_date, "%Y-%m-%d")
t_end = datetime.strptime(end_date, "%Y-%m-%d")

num_months = max(1, (t_end.year - t_start.year) * 12 + t_end.month - t_start.month + 1)
month_dates = [t_start + relativedelta(months=i) for i in range(num_months)]
for date in month_dates:
    samples = []
    for i in range(((date + relativedelta(months=1)) - date).days):
        search_date = date + timedelta(days=i)
        if search_date > t_end:
            continue
        search = Fido.search(
            a.Time(search_date, search_date + timedelta(days=1)),
            a.Provider("SSC"),
            a.Instrument("EUVI"),
            a.AttrOr([a.Wavelength(wav*u.AA) for wav in wavelengths]),
        )
        break

In [ ]:
len(search)

4

In [ ]:
search_171 = search[0]
search_195 = search[1]
search_284 = search[2]
search_304 = search[3]

In [ ]:
stereoA_171 = search_171[(search_171['Source'] == 'STEREO_A')]
stereoA_195 = search_195[(search_195['Source'] == 'STEREO_A')]
stereoA_284 = search_284[(search_284['Source'] == 'STEREO_A')]
stereoA_304 = search_304[(search_304['Source'] == 'STEREO_A')]

stereoB_171 = search_171[(search_171['Source'] == 'STEREO_B')]
stereoB_195 = search_195[(search_195['Source'] == 'STEREO_B')]
stereoB_284 = search_284[(search_284['Source'] == 'STEREO_B')]
stereoB_304 = search_304[(search_304['Source'] == 'STEREO_B')]


print("STEREO-A 171:", len(stereoA_171))
print("STEREO-A 195:", len(stereoA_195))
print("STEREO-A 284:", len(stereoA_284))
print("STEREO-A 304:", len(stereoA_304))
print()
print("STEREO-B 171:", len(stereoB_171))
print("STEREO-B 195:", len(stereoB_195))
print("STEREO-B 284:", len(stereoB_284))
print("STEREO-B 304:", len(stereoB_304))

STEREO-A 171: 80
STEREO-A 195: 287
STEREO-A 284: 30
STEREO-A 304: 163

STEREO-B 171: 12
STEREO-B 195: 287
STEREO-B 284: 12
STEREO-B 304: 144


In [ ]:
stereoA_171_dates = stereoA_171['Start Time']
stereoA_195_dates = stereoA_195['Start Time']
stereoA_284_dates = stereoA_284['Start Time']
stereoA_304_dates = stereoA_304['Start Time']

stereoB_171_dates = stereoB_171['Start Time']
stereoB_195_dates = stereoB_195['Start Time']
stereoB_284_dates = stereoB_284['Start Time']
stereoB_304_dates = stereoB_304['Start Time']

print("STEREO-A 171:", stereoA_171_dates[0])
print("STEREO-A 195:", stereoA_195_dates[0])
print("STEREO-A 284:", stereoA_284_dates[0])
print("STEREO-A 304:", stereoA_304_dates[0])
print()
print("STEREO-B 171:", stereoB_171_dates[0])
print("STEREO-B 195:", stereoB_195_dates[0])
print("STEREO-B 284:", stereoB_284_dates[0])
print("STEREO-B 304:", stereoB_304_dates[0])

STEREO-A 171: 2011-01-01 00:14:00.000
STEREO-A 195: 2011-01-01 00:05:30.000
STEREO-A 284: 2011-01-01 00:16:30.000
STEREO-A 304: 2011-01-01 00:06:15.000

STEREO-B 171: 2011-01-01 00:14:46.000
STEREO-B 195: 2011-01-01 00:06:16.000
STEREO-B 284: 2011-01-01 00:17:16.000
STEREO-B 304: 2011-01-01 00:07:01.000


In [ ]:
from datetime import datetime, timedelta
from sunpy.net import Fido, attrs as a

t_start = datetime.strptime(start_date, "%Y-%m-%d")
t_end = datetime.strptime(end_date, "%Y-%m-%d")
td = timedelta(days=1)
cadence = get_astropy_time("1 days")
date_list = [t_start + i * td for i in range((t_end - t_start) // td)]

for d in date_list:
    stereo_a = Fido.search(
        a.Time(d, d + td),
        a.Instrument("EUVI"), 
        a.Source('STEREO_A'), 
        a.AttrOr([a.Wavelength(wl*u.AA) for wl in wavelengths]),
        a.Sample(cadence)
    )
    if d > datetime(2014, 10, 1): # loss of STEREO B
        continue
    stereo_b = Fido.search(
        a.Time(d, d + td),
        a.Instrument("EUVI"), 
        a.Source('STEREO_B'), 
        a.AttrOr([a.Wavelength(wl*u.AA) for wl in wavelengths]),
        a.Sample(cadence)
    )

In [ ]:
stereoA_171 = stereo_a[0]
stereoA_195 = stereo_a[1]
stereoA_284 = stereo_a[2]
stereoA_304 = stereo_a[3]

stereoB_171 = stereo_b[0]
stereoB_195 = stereo_b[1]
stereoB_284 = stereo_b[2]
stereoB_304 = stereo_b[3]

print("STEREO-A 171:", len(stereoA_171), stereoA_171['Start Time'][0], stereoA_171["Wavelength"][0], stereoA_171["Source"][0])
print("STEREO-A 195:", len(stereoA_195), stereoA_195['Start Time'][0], stereoA_195["Wavelength"][0], stereoA_195["Source"][0])
print("STEREO-A 284:", len(stereoA_284), stereoA_284['Start Time'][0], stereoA_284["Wavelength"][0], stereoA_284["Source"][0])
print("STEREO-A 304:", len(stereoA_304), stereoA_304['Start Time'][0], stereoA_304["Wavelength"][0], stereoA_304["Source"][0])
print()
print("STEREO-B 171:", len(stereoB_171), stereoB_171['Start Time'][0], stereoB_171["Wavelength"][0], stereoB_171["Source"][0])
print("STEREO-B 195:", len(stereoB_195), stereoB_195['Start Time'][0], stereoB_195["Wavelength"][0], stereoB_195["Source"][0])
print("STEREO-B 284:", len(stereoB_284), stereoB_284['Start Time'][0], stereoB_284["Wavelength"][0], stereoB_284["Source"][0])
print("STEREO-B 304:", len(stereoB_304), stereoB_304['Start Time'][0], stereoB_304["Wavelength"][0], stereoB_304["Source"][0])

STEREO-A 171: 1 2011-01-01 00:14:00.000 [171. 175.] Angstrom STEREO_A
STEREO-A 195: 1 2011-01-01 00:05:30.000 [195. 195.] Angstrom STEREO_A
STEREO-A 284: 1 2011-01-01 00:16:30.000 [284. 284.] Angstrom STEREO_A
STEREO-A 304: 1 2011-01-01 00:06:15.000 [304. 304.] Angstrom STEREO_A

STEREO-B 171: 1 2011-01-01 00:14:46.000 [171. 175.] Angstrom STEREO_B
STEREO-B 195: 1 2011-01-01 00:06:16.000 [195. 195.] Angstrom STEREO_B
STEREO-B 284: 1 2011-01-01 00:17:16.000 [284. 284.] Angstrom STEREO_B
STEREO-B 304: 1 2011-01-01 00:07:01.000 [304. 304.] Angstrom STEREO_B
